simaliar to the first attempt but with cleaner code, more flexible layer creation

In [ ]:
import numpy as np
import math
import copy
print("start!")


In [ ]:
def create_layer(layer_size, dtype=np.float32, **kwargs):
    layer = np.zeros(layer_size, dtype=dtype)
    return(layer)

def create_layer_weights(prev_ly_size, curr_ly_size, init=True):
    if init:
        ly_weights = np.random.rand(prev_ly_size, curr_ly_size)
    else:
        ly_weights = np.zeros(shape=(prev_ly_size, curr_ly_size))
    return(ly_weights)

def create_biases(ly_size, init=True):
    if init:
        biases = np.random.rand(ly_size)
    else:
        biases = np.zeros(ly_size)
    return(biases)


def label_to_output(int):
    outlayer = np.zeros(shape=(10), dtype=np.float32)
    outlayer[int] = 1
    return(outlayer)

class n_network:
    def __init__(self, layers, biases, weights):
        self.layers = layers
        self.biases = biases
        self.weights = weights

def create_network(in_ly_size, out_ly_size, hidden_ly_num, hidden_ly_size):
    in_layer = create_layer(in_ly_size)
    hidden_lys = []
    for index in range(hidden_ly_num):
        hidden_lys.append(create_layer(hidden_ly_size))
    out_layer = create_layer(out_ly_size)
    network_lys = [in_layer, *hidden_lys, out_layer]


    biases =[]
    for i in range(1, len(network_lys), 1):
        biases.append(create_biases(np.size(network_lys[i]), init=False))

    weights = []
    for i in range(1, len(network_lys), 1):
        weights.append( create_layer_weights(np.size(network_lys[i-1]), np.size(network_lys[i]), init=True))

    network = n_network(network_lys, biases, weights)
    return(network)


def sigx(number):
    #sigmond function
    try:
        sig_x = ( 1 / (1+ math.exp(-number)))
        #this does not woprk for large negitive numbers 
        # due to rounding errors leading to devide by zero so we'll add error handling
    except OverflowError:
        sig_x = 0
    return(sig_x)

def forward(input_vals:np.ndarray, net:n_network):
    #first layer
    input_vals = np.array(input_vals)
    net.layers[0] = input_vals

    for index in range(1, len(net.layers), 1):
        net.layers[index] = np.dot(net.layers[(index-1)], net.weights[index-1]) + net.biases[index-1]
        
        #activation function
        sigVec = np.vectorize(sigx)
        net.layers[index] = sigVec(net.layers[index])
    return(net)

def node_delta(expected:np.array, actual:np.array):
    delta = expected - actual
    return(delta)

def deep_copy_mat_list (list_to_copy, propigate_cell_vals=True):
    #creates a copy of a list containing numpy arrays of varible size, ethier as empty arrays or with the same values
    
    new_list = []
    if propigate_cell_vals:
        for items in list_to_copy:
            new_list.append(items)
    else:
        for items in list_to_copy:
            new_list.append(np.zeros_like(items))       

    return(new_list)



def loss(network_output_ly, expected_ly):
    #loss =  mean of all output neurons - the expected, squared
    loss_vec = (network_output_ly - expected_ly)**2
    loss_sclr = np.sum((network_output_ly - expected_ly)**2) / len(network_output_ly)
    return(loss_sclr, loss_vec)

def backprop_single(net: n_network, expected_out_ly):
    #does backprop for a single training example, duhh
    #add learning rate??
    
    expected_out_ly = np.array(expected_out_ly)
    w_delta = deep_copy_mat_list(net.weights, propigate_cell_vals=False)
    b_delta = deep_copy_mat_list(net.biases, propigate_cell_vals=False)
    a_delta = deep_copy_mat_list(net.layers, propigate_cell_vals=False)

    first = True
    for l in range(len(net.layers)-1, 0, -1):
        print("at l of:", l)

        if first:
            #first(last) layer
            first = False
            a_delta_dir = expected_out_ly - net.layers[l]  
            w_delta[l-1] = np.outer(net.layers[l-1], a_delta_dir)   
            a_delta[l] = a_delta_dir
        else:
            a_delta_dir = a_delta[l] - net.layers[l]
            w_delta[l-1] = np.outer(net.layers[l-1], a_delta_dir)   

        a_delta[l-1] = np.dot(net.weights[l-1], net.layers[l])
        a_delta[l-1] = a_delta[l-1] / a_delta[l-1].size
    return(w_delta, b_delta, a_delta)




In [ ]:
net = create_network(3,5,1,2)
input = [1,1,1]
output = [0,1,0,1,0]

net.weights[1] = np.transpose(np.array([[0,0],
                           [0.5,0.5],
                           [0.3,-1],
                           [0.3,-1],
                           [0.1,0.1]]))
net.weights[0] = (np.array([[1,1],
                           [0.5,0.5],
                           [0.3,-1]]))
print(np.shape(net.weights[1]))

net = forward(input,net)
for index, items in enumerate(net.layers):
    print("layer: ", index)
    print("net layer size: ",np.shape(items), "\nand layer values: ", items)
for index, items in enumerate(net.weights):
    print("layer: ", index)
    print("net weight layer size: ",np.shape(items), "\nand weight layer values: ", items)
for index, items in enumerate(net.biases):
    print("layer: ", index)
    print("net bias layer size: ",np.shape(items), "\nand bias layer values: ", items)


deltas_list = backprop_single(net, output)
for index, items in enumerate(deltas_list[0]):
    print("layer: ", index)
    print("Weight change layer size: ",np.shape(items), "\nand weight change values: ", items)
for index, items in enumerate(deltas_list[1]):
    print("layer: ", index)
    print("bias change layer size: ",np.shape(items), "\nand bias change values: ", items)
for index, items in enumerate(deltas_list[2]):
    print("layer: ", index)
    print("activation change layer size: ",np.shape(items), "\nand activation change values: ", items)


In [ ]:
network = create_network(3,3,1,3)

input = [1,1,2]
output = [0,1,0]


network = forward(input,network)
#print("layers: ")
#print(network.layers[:])
print("weights: ")
for bunchs in network.weights:
    print(bunchs)
w_delta = np.zeros_like(network.weights)
print("weights zeros: ")
print("wdelta: ", w_delta)
print("wdelta for loop: ")
for bunchs in w_delta:
    print(bunchs)

#print("ouput:",network.layers[-1])

#print(loss(network.layers[-1], output))

print(network.layers[1])

print(network.layers[1])